In [1]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch

from train_dynamics import generate_data

In [2]:
device = "cpu"
cached_fn = os.path.join("dynamics_trained", "dynamics.nf")
with open(cached_fn, "rb") as f:
    encoder = pickle.load(f)
encoder.to(device)

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=15, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=12, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=87, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=15, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_laye

In [3]:
def calc_scores(xs, Cs, k=10):
    cal_samples = encoder.sample(k, xs).detach().cpu().numpy()
    cal_samples = cal_samples.reshape((cal_samples.shape[0], k, 2, 3))
    cal_tiled = np.transpose(np.tile(Cs.detach().cpu().numpy(), (k, 1, 1, 1)), (1, 0, 2, 3))
    cal_diff = cal_samples - cal_tiled
    cal_norms = np.linalg.norm(cal_diff, ord=2, axis=(2,3))
    return np.min(cal_norms, axis=-1)

In [6]:
# C in R^2x3
cal_sims = 1_000
cal_C_hats, cal_x = generate_data(cal_sims)
cal_C_hats = cal_C_hats.reshape((cal_sims, 2, 3))
cal_scores = calc_scores(cal_x, cal_C_hats)
q_hat = np.quantile(cal_scores, q = 0.95)

In [8]:
k = 10
test_sims = 1
_, test_x = generate_data(test_sims)
test_samples = encoder.sample(k, test_x).detach().cpu().numpy()
test_samples = test_samples.reshape((test_samples.shape[0], k, 2, 3))

In [ ]:
# two steps:
# 1. policy gradient for *fixed C*: check if formulation is correct (compare to C obtained from Ricatti eqns)
# --- attempt w/ our choice of A/B (w/out noise)
# 2. find C maximizer: compare to alt. robust formulations
# 3. done with synthetics?